### 터미널에서 Kubectl 설치 필요
```
sudo apt-get update && sudo apt-get install -y apt-transport-https gnupg2
curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://apt.kubernetes.io/ kubernetes-xenial main" | sudo tee -a /etc/apt/sources.list.d/kubernetes.list
sudo apt-get update
sudo apt-get install -y kubectl=1.16.9-00
```

### 전체 파드 조회가 필요하므로 .kube/config 설정 필요
```
vi .kube/config
// cto config 설정 //
```

In [81]:
import os
import requests
import json
import pandas as pd
import subprocess

from spellchecker import SpellChecker

In [82]:
prom_query_url = 'http://13.125.63.249:30915/api/v1/query?query='

In [83]:
def call_data(url, query):
    r = requests.get(url + query)
    return r

In [84]:
url_prom = 'http://13.125.63.249:30915/api/v1/query?query='
query = 'ALERTS{alertname="KubeDeploymentReplicasMismatch"}'
json.loads(call_data(url_prom, query).text)


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'ALERTS',
     'alertname': 'KubeDeploymentReplicasMismatch',
     'alertstate': 'firing',
     'deployment': 'test-mysql',
     'endpoint': 'http',
     'instance': '10.220.105.27:8080',
     'job': 'kube-state-metrics',
     'namespace': 'test-alerts',
     'pod': 'prom-kube-state-metrics-76b6bbdb5b-dpvdk',
     'service': 'prom-kube-state-metrics',
     'severity': 'critical'},
    'value': [1600748600.972, '1']}]}}

In [85]:
def kube_deployment_replicas_mismatch():
    ## Alerts case : KubeContainerWaiting
    alert_query = 'ALERTS{alertname="KubeDeploymentReplicasMismatch"}'
    alert_get = call_data(prom_query_url, alert_query)
    alert_message = json.loads(alert_get.text)
    
    fix_list = []
    for item in alert_message['data']['result']:
        #print(item)
        check_list = {'namespace':item['metric']['namespace'], 'deployment':item['metric']['deployment']}
        #check_list = {item['metric']['namespace'], item['metric']['pod']}
        print(check_list)        
        fix_list.append(check_list)     
    
    return(fix_list) 

In [86]:
kube_deployment_replicas_mismatch()

{'namespace': 'test-alerts', 'deployment': 'test-mysql'}


[{'namespace': 'test-alerts', 'deployment': 'test-mysql'}]

In [87]:
def kubectl_jsonpath():
    pending_data = kube_deployment_replicas_mismatch()
    cnt = len(pending_data)    
    df_data = pd.DataFrame(pending_data)
    #print(df_data)
    
    kubectl_result = []
    for i in range(cnt):
        namespace = df_data.namespace[i]
        deployment = df_data.deployment[i]
        
        # !kubectl -n aiops get pods test-pending -o jsonpath='{.status.containerStatuses[*].state.waiting.message}' | awk -F"\"" '{print $2}'
        
        #
        #cli = "kubectl -n " + namespace + " get deployments " + deployment + \
        #      " -o jsonpath='{.status.containerStatuses[*].state.waiting.message}'" + \
        #      " | awk -F'\"' '{print $2}'"
        #
        cli = "kubectl -n " + namespace + " get deployments " + deployment + \
              " -o jsonpath='{.spec.template.spec.containers[*].name}'" 
        
        kubectl_command_line = subprocess.check_output(cli, shell=True,  encoding='utf-8')
        print("deployment 이름 : " + deployment + " 현재 적용된 이미지 : " + kubectl_command_line)
        tmp_string = namespace + " " + deployment + " " + kubectl_command_line
        
        ### sort :: namespace pod result        
        kubectl_result.append(tmp_string)
        
    return(kubectl_result)
    


In [88]:
kubectl_jsonpath()

{'namespace': 'test-alerts', 'deployment': 'test-mysql'}
deployment 이름 : test-mysql 현재 적용된 이미지 : mysqll


['test-alerts test-mysql mysqll']

In [109]:
def recommand_image():
    pending_images = kubectl_jsonpath()
    cnt = len(pending_images)
    spell = SpellChecker()
    spell.word_frequency.load_text_file('./words.txt')
    
    #result = []
    for i in range(cnt):
        namespace = pending_images[i].split(' ')[0]
        deployment = pending_images[i].split(' ')[1]   
        m_image = pending_images[i].split(' ')[2]
        
        #misspelled = spell.unknown([m_image])
        tmp_recommend = spell.correction(m_image)
        tmp_string = "namespace: " + namespace \
                   + " , deployment: " + deployment \
                   + "\n" + "   시스템이 " + m_image + " 이미지 대신 " + tmp_recommend + " 이미를 추천합니다." \
                   + "\n" + "아래 명령 라인을 이용해서 수정 가능합니다. " \
                   + "\n" + "kubectl -n " + namespace + " edit deployments " + deployment
        
        result = tmp_string
        print(result)
        
    #return(result)
    

In [110]:
if __name__=="__main__":
    #print(kube_container_waiting())
    #print(kubectl_jsonpath())
    print(recommand_image())
    

{'namespace': 'test-alerts', 'deployment': 'test-mysql'}
deployment 이름 : test-mysql 현재 적용된 이미지 : mysqll
namespace: test-alerts , deployment: test-mysql
   시스템이 mysqll 이미지 대신 mysql 이미를 추천합니다.
아래 명령 라인을 이용해서 수정 가능합니다. 
kubectl -n test-alerts edit deployments test-mysql
None
